In [1]:
%matplotlib widget

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyodbc

In [3]:
font = {'family' : 'serif',
        'serif' : 'Times New Roman',
        'size'   : 8}
plt.rc('font', **font)

In [4]:
server = 'mrwhite.intrans.iastate.edu'
database = 'sim_Andalib'

cnxn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes')

In [5]:
def drop_first_row(sig_df):

    if (sig_df.iloc[0][1] == 1 or sig_df.iloc[0][1]== sig_df.iloc[1][1]): 
        sig_df = sig_df.iloc[1:, :].copy()
        return drop_first_row(sig_df)
    else: return sig_df
    
    

In [6]:
def pcd_pog(signal_df, veh_trajectory):
    signal_df = drop_first_row(signal_df)
#     breakpoint()
    signal_df.loc[:, 'cycle'] = np.hstack([0, signal_df.iloc[:-1, 2]]) ### column 2 = eog
    signal_df.loc[:, 'x_axis'] = signal_df.bog+np.hstack([signal_df.iloc[1:, 3], 0]) ### column 3 = bog, 86400 = 3600*24
    signal_df.loc[:, 'bog2'] = signal_df.bog - signal_df.cycle
    signal_df.loc[:, "bog2"] = signal_df["bog2"] + np.hstack([0,0,signal_df.iloc[1:-1, 6]])### column 6  = bog2

    eog_2 = np.array(signal_df.iloc[1:, 2]) - np.array(signal_df.iloc[:-1, 4])
    eog_2[2:] += eog_2[1:-1] 
    eog_2 = np.hstack([eog_2, eog_2[-1]])

    signal_df.loc[:, 'eog2'] = eog_2

    eog_list = signal_df.loc[signal_df["eog"] != 0, "eog"]
    bog_list = signal_df.loc[signal_df["bog"] != 0, "bog"]

    veh_trajectory["timeStamp"] = veh_trajectory.time - veh_trajectory.travel_time + travel_time_intersection
    veh_trajectory = veh_trajectory.sort_values(by=['timeStamp'])
    veh_trajectory = pd.merge_asof(veh_trajectory, eog_list,left_on='timeStamp', right_on='eog', direction = 'backward')
    veh_trajectory["TIC"] = veh_trajectory.timeStamp - veh_trajectory.eog
    veh_on_green = pd.merge_asof(veh_trajectory, bog_list,left_on='eog', right_on='bog', direction = 'forward')
    veh_on_green = veh_on_green[veh_on_green.TIC>=(veh_on_green.bog-veh_on_green.eog)]

    pog = len(veh_on_green[(veh_on_green.timeStamp>=simulation_start) & (veh_on_green.timeStamp<=simulation_end)])
    pog /= len(veh_trajectory[(veh_trajectory.timeStamp>=simulation_start) & (veh_trajectory.timeStamp<=simulation_end)])
        
    return signal_df[['x_axis','bog2', 'eog2']], veh_trajectory[['timeStamp', 'TIC']],veh_on_green[['timeStamp', 'TIC']], pog

In [7]:
# def extension_plot(signal_df, extension_calls, sc, sg):
#     sig_df = signal_df.drop_duplicates(['x_axis'], keep = 'last').copy()
#     sig_df['x2'] = np.hstack([sig_df.iloc[1:, 0], simulation_end])
    
#     ext_call_df = extension_calls.loc[(extension_calls.SC == sc) & (extension_calls.SG == sg)].copy()
#     ext_call_df = ext_call_df.drop(['SC', 'SG'], axis = 1)
#     ext_call_df.sort_values(['simSec'], inplace = True)
#     ext_df = pd.merge_asof(ext_call_df, sig_df, right_on='x_axis', left_on = 'simSec', direction = 'backward')
#     ext_df2 = ext_df.drop(['eog2', 'max_green', 'bog'], axis = 1)
#     ext_df2['ext_call'] = ext_df2.bog2 + ext_df2.tSigState
#     ext_df2['width'] = ext_df2.x2 - ext_df2.x_axis
    
#     return ext_df2[['x_axis', 'ext_call','width', 'extension_time']]

In [13]:
simulation_start = 300
simulation_end = 3600
travel_time_intersection = 5

sim_id = 21
iteration = 1

In [8]:
titles = ["Zero-offset", "AOG" , "Modified-AOG", "AOG-CV", "Delay" , "Number of Stops" , "Delay + Stops", "PrOG", "Kell Method", "BW" , "Maxband", "Multiband" ]
run_ids = [1, 2,3,5,4,6,12, 10, 16, 8, 15, 14]

In [20]:
signal_sql_test = "SELECT [StudyID], [RunID], [SimTime],[SC], [SG], [state] FROM [sim_Andalib].[dbo].[signalchanges] where [StudyID]=" \
                    + str(sim_id)+"and [Iteration]="+str(iteration)+ "and [SC]>= 9005 and [SC]<= 9012 and [SG]=" + str(sg_) \
                    + "Order By [SimTime]"

signal = pd.read_sql(signal_sql_test, cnxn)

if(len(signal)==0): print("ERROR")
else:
    signal = signal.loc[signal.state.isin([0, 1])]
    signal["eog"] = signal.SimTime - signal.SimTime*signal.state
    signal["bog"] = signal.SimTime - signal.SimTime*(1-signal.state)

In [21]:
delay_sql = "SELECT [RunID], [Time],[No_],[Veh],[Trav_] FROM [sim_Andalib].[dbo].[traveltimes] where [StudyID]=" \
            + str(sim_id) +"and [Iteration]=" + str(iteration) 

delay = pd.read_sql(delay_sql, cnxn)
delay.columns = ['RunID', 'time','tt_id', 'vehNo', 'travel_time']
delay.sort_values(by=['tt_id', 'time'], inplace = True)

In [28]:
# sc_ = [9012,9011,9010,9009, 9008, 9007,9006,9005]
# sg_ = 6
# tt_ = [31204, 31104, 31004, 30904,30804, 30704, 30604, 30504]

sc_ = [9005,9006,9007,9008, 9009, 9010,9011,9012]
sg_ = 2
tt_ = [30504, 30604, 30704, 30804,30904, 31004, 31104, 31204]

# sc_ = [9007, 9008, 9009]
# sg_ = 2
# tt_ = [ 30703,30803, 30903]

# sc_ = [9009, 9008, 9007]
# sg_ = 6
# tt_ = [ 30904,30804, 30704]


assert(len(sc_)==len(tt_))

figure_size = (15, 8.5)


In [17]:
directory = "C:\\Users\\ashams\\Box\\research works\\comparison signal offset optimization\\analysis\\"

In [29]:
fig, axes = plt.subplots(nrows = 12, ncols = 8,sharey = True, sharex = True, figsize=figure_size)

fig.tight_layout()
fig.subplots_adjust(wspace=0.05, hspace=0.1)

for  i in range(len(titles)):
    for j in range(len(sc_)):
        
        sc_num = sc_[j]
        tt_num = tt_[j]
        run_id = run_ids[i]
        
        print(sc_num, " ", tt_num)
#         i = itr//2
#         j = itr%2

        signal_df = signal.loc[(signal.RunID == run_id) & (signal.SC == sc_num), ['SimTime', 'state', 'eog', 'bog']].copy()
        traj_df = delay.loc[(delay.RunID == run_id) & (delay.tt_id == tt_num),['time', 'travel_time']].copy()

        signal_df, veh_arrival, veh_on_green, pog =  pcd_pog(signal_df, traj_df)

        if(signal_df.iloc[-1][0]==0):    signal_df.iloc[-1][0]=simulation_end

        pcd = axes[i][j]
        pcd.plot(signal_df['x_axis'], signal_df['bog2'], color = 'g')
        pcd.plot(signal_df['x_axis'], signal_df['eog2'], color = 'r')
        pcd.fill_between(signal_df.x_axis, signal_df.bog2, signal_df.eog2, color = 'g', alpha = 0.5)
        pcd.scatter(veh_arrival["timeStamp"], veh_arrival["TIC"], color = 'black', marker = '.', s = 0.2)

        pcd.set_xlim([simulation_start, simulation_end])
        pcd.set_ylim([0,150])
#         pcd.set_yticklabels([0, 50, 100])
        pcd.grid(which = 'both', linestyle = 'dashed')
        pcd.grid(b=True, which='minor', alpha=0.2)
        pcd.minorticks_on()
#         pcd.set_xlabel("")
#         pcd.set_ylabel("")
        if(i==0): pcd.set_title("Int. "+ str(sc_num - 9004))
        if(j==0): pcd.set_ylabel(titles[i])
        pcd.text(.5,.85, "POG:" + str(round(pog*100, 2)), fontsize = 9,horizontalalignment='center', transform=pcd.transAxes)

    
fig.text(-0.05, 0.45, 'Cycle Second',ha='center', rotation = 90, fontsize = 11)    
fig.text(0.5, 0.0, 'Simulation Second', ha='center', fontsize = 11)

fig.savefig(directory+"pcd_eb_all.png", dpi = 300,  bbox_inches='tight')
# plt.close(fig)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

9005   30504
9006   30604
9007   30704
9008   30804
9009   30904
9010   31004
9011   31104
9012   31204
9005   30504
9006   30604
9007   30704
9008   30804
9009   30904
9010   31004
9011   31104
9012   31204
9005   30504
9006   30604
9007   30704
9008   30804
9009   30904
9010   31004
9011   31104
9012   31204
9005   30504
9006   30604
9007   30704
9008   30804
9009   30904
9010   31004
9011   31104
9012   31204
9005   30504
9006   30604
9007   30704
9008   30804
9009   30904
9010   31004
9011   31104
9012   31204
9005   30504
9006   30604
9007   30704
9008   30804
9009   30904
9010   31004
9011   31104
9012   31204
9005   30504
9006   30604
9007   30704
9008   30804
9009   30904
9010   31004
9011   31104
9012   31204
9005   30504
9006   30604
9007   30704
9008   30804
9009   30904
9010   31004
9011   31104
9012   31204
9005   30504
9006   30604
9007   30704
9008   30804
9009   30904
9010   31004
9011   31104
9012   31204
9005   30504
9006   30604
9007   30704
9008   30804
9009   30904

In [15]:
# extension_calls = pd.read_csv(directory+'\\'+str(sensing_range)+'\\extension.csv', header = None, delimiter = ';')
# extension_calls.columns = ['SC', 'SG', 'simSec', 'tSigState', 'extension_time', 'vc_ratio', 'Num_of_vehicles', 'reason']
# extension_calls = extension_calls.loc[extension_calls.extension_time>0.15]

# extension_calls['bog'] = extension_calls.simSec - extension_calls.tSigState + 0.1

# extension_calls['SC'] = extension_calls['SC'].astype(int)
# extension_calls['SG'] = extension_calls['SG'].astype(int)
# extension_calls['SC'] = extension_calls['SC']+1
# extension_calls.loc[extension_calls.SG==1, 'SG'] = 2
# extension_calls.loc[extension_calls.SG==5, 'SG'] = 6 

# ext_df = extension_plot(signal_df, extension_calls, 3, 2)
# for ind in range(len(ext_df)):
#     pcd.add_patch(patches.Rectangle((ext_df.iloc[ind][0], ext_df.iloc[ind][1]), ext_df.iloc[ind][2], ext_df.iloc[ind][3], alpha = 0.6))